# Quadratic Sieve Implementation

## Preliminaries

Imports

In [4]:
import math
import numpy as np

### Part 1.1: Strong Pseudoprime Test

A number of modules (below) require a means to quickly take some exponent mondulo some number...  
Notes on Fast Modular Exponentiation:  
* Use the property a * b mod (n) = a mod (n) * b mod (n)  
* Recursively compute: base * (previous remainder) mod (n) over exponent => 1 : exponent

In [5]:
# FAST MODULAR EXPONENTIATION
# b = base for exponentiation
# exp = exponent
# n = modulo
# returns result of fast modular exponentiation
def fastExp(b, t, n):
    res = 1
    for i in range(0, t):
        res = (res * b) % n
    # Check result for strong pseudoprime context
    if (res == (n - 1)):
        return -1
    return res

In [6]:
print(fastExp(11, 13, 19))

11


A few notes to consider when implementing the strong pseudoprime test:  
* There are no strong Carmichael Numbers - this was shown in class and homework  
* A composite number n is a strong pseudoprime to at most one quarter of all bases below n  

So, the test will be iterated over a number of bases to ensure (very high probability) primality  
For the test, there is no composite that passses the test for 2, 325, 9375, 28178, 450775, 9780504, and 1795265022  
* https://en.wikipedia.org/wiki/Strong_pseudoprime#cite_note-10  



In [7]:
# STRONG PSEUDOPRIME TEST
# n = questionable prime
# bases = array of bases for test
# returns 0 on passed, -1 on failed test
def strongPseudo(n, bases):
    for i in range(len(bases)):
        t = n - 1
        while (not(t % 2)):
            res = fastExp(bases[i], t, n)
            if (res == 1):
                t = t // 2
                # Continue base checking
                continue
            elif (res == -1):
                # Assume probable pseudoprime of current base
                break
            else:
                # Number is composite - fails test 
                return -1
    return 0

n = 3004879 
bases = [2, 325, 9375, 28178]
# print(fastExp(3, 3962, 31697))
# NOTE: Write a loop to exemplify 

print(strongPseudo(n, bases))

-1


### Part 1.2: Trial Division

Notes on Trial Division:  
* Trial Division is used to factor out primes of some number, n, up to some max value.  
* The unfactored portion (after max) will be called 's'  
* The result of this module will be used in finding a root modulo p (Tonelli's Algorithm)

In [8]:
# TRIAL DIVISION
# n = Number to divide
# parr = array of primes 
# returns arr (trial prime factorization exponents), n (AKA 's' the unfactored portion)
def trialDiv(n, parr):
    arr = [0] * len(parr)
    # Check each prime in prime array input
    for i in range(len(parr)):
        # Keep dividing prime out until no longer can
        while (True):
            if (n % parr[i] == 0):
                arr[i] = arr[i] + 1
                n = n // parr[i]
            else:
                break
    return arr, n

# NOTE: Write loop to exemplify 
print(trialDiv(31696, [2]))

([4], 1981)


### Part 2.1: Inverse Modulo

In Tonelli's algorithm (see 2.2) 
Notes on Inverse Modulo:  
* Implement Extended Euclidean Algorithm for finding some inverse of a modulo n  


In [9]:
# EXTENDED EUCLIDEAN ALGORITHM
def extEuclid(a,b):
    n = b
    x_prime = 1
    x = 0
    #prevx, x = 1, 0
    while b:
        q = a // b
        # Update current iteration of inverse
        temp = x
        x = x_prime - (q * x)
        x_prime = temp
        # Update a & b
        temp1 = a
        a = b
        b = temp1 % b
    if (x_prime < 0):
        return n + x_prime
    return x_prime

print(extEuclid(8, 193))

169


In [10]:
e = pow(2, 5) * 3
a = pow(8, 3)
print(a)
print(extEuclid(a, 193))

512
72


In [17]:
print(fastExp(8, 3, 193))

126


### Part 2.2: Tonelli's Algorithm 

Calculating Jacobi symbols will be necessary for this module  
Notes on Jacobi Symbols:  
* j

In [10]:
# JACOBI SYMBOLS
# n = integer in question
# p = prime modulus
# Returns 1 (on residue exists), -1 (on non-residue), 0 (p divides n)
def jacobi (a, n):
    j = 1
    # Main Loop
    while a != 0:
        # Loop for pulling out factors of two
        while a % 2 == 0:
            a = a // 2
            if ((n % 8 == 3) or (n % 8 == 5)):
                j = j * -1
        # Swap a, n
        temp = a
        a = n
        n = temp
        if ((a % 4 == 3) and (n % 4 == 3)):
            j = j * -1
        a = a % n
    if n == 1:
        return j
    return 0

a = [2, 3, 5, 7, 11, 13, 17, 19, 23, 29]
n = 9991
for i in a:
    print("Jacobi Symbol {} / {} = {}".format(n, i, jacobi(n, i)))

Jacobi Symbol 9991 / 2 = 1
Jacobi Symbol 9991 / 3 = 1
Jacobi Symbol 9991 / 5 = 1
Jacobi Symbol 9991 / 7 = 1
Jacobi Symbol 9991 / 11 = 1
Jacobi Symbol 9991 / 13 = -1
Jacobi Symbol 9991 / 17 = -1
Jacobi Symbol 9991 / 19 = 1
Jacobi Symbol 9991 / 23 = 1
Jacobi Symbol 9991 / 29 = -1


Notes on Tonelli's Algorithm:  
This method is used for finding a root mod some prime, which will be very useful in the Quadratic Sieve   
* The following example detatils the steps  
  * Take x^2 = 9 mod 73  
  1. Check if 9 is a quadratic residue of 73 (via Jacobi Symbols)
  2. Use Trial Division (see 1.2) to obtain some s, t pair  
      * 73 - 1 = 72, 72 = 2^3 * 9 (s = 3, t = 9)  
  3. Find a non-residue (b mod p) via repeated jacobi symbols ((b/p) = -1) (b is new base - see below)
  4. Find an (even) i which satisfies (EQ: (a/(b^i))^(2^(s-k)*t) = 1 mod p), where 'a' is the square, 'p' is the prime, and 'b' is the base of i
  * b (inv)^i mod p  - > b (inv p) -> a * ans ^i -> ans ^ e -> check 1 mod p
      * Start with i = 2
      * Check if ik satisfies the EQ
            * If it does satisfy for remainder 1, then i(k+1) = i(k)    
             * O.W. i(k+1) = i(k) + 2^k
      * The above is computed for k = 1 -> s (see part 2)
  5. Take the final i(k = s) and compute: (b^(i/2)) * ((a/(b^i))^((t+1)/2)) mod p
  6. Pray 

In [12]:
# TONELLI'S ALGORITHM 
# a = number 
# n = modulus 
# Returns a root mod n (r), o.w. -1
def tonelli(a, p):
    # Check Jacobi Symbol (a/p) = 1
    if (jacobi(a, p) != 1):
        print("{} is not a quadratic residue mod {}".format(a, p))
        return -1
    # Get s, t
    s_arr, t = trialDiv(p - 1, [2])
    s = s_arr[0]
    # Find a non-residue 
    b = 2
    while (True):
        if (jacobi(b, p) == -1):
            break
        b = b + 1
    # Finding ik
    i_arr = []
    i = 2
    k = 1
    while (k <= s):
        print("i = {}, k = {}".format(i, k))
        e = pow(2, (s-k)) * t
        alpha = pow(a, e)
        # Get inverse a mod p
        if ((fastExp(a, e, p)) == fastExp(b, (i*e), p)):
        # if (extEuclid(a, p) == pow(b, (i))):
            # Current i works, append and continue
            i_arr.append(i)
            k = k + 1
            continue
        # Current i doesnt work, neeed to update
        # NOTE: One of the i values must work, so no need to check
        i = i + pow(2, k)
        i_arr.append(i)
        k = k + 1
    print(i_arr)
    # Compute final root
    return (fastExp(b, (i//2), p) * fastExp(a, ((t+1)//2), p))

print(tonelli(8, 193))



i = 2, k = 1
i = 2, k = 2
i = 2, k = 3
i = 10, k = 4
i = 26, k = 5
i = 58, k = 6
[2, 2, 10, 26, 58, 122]
8640


## Main Modules for Sieving

### Part 3: Gaussian Elimination Modulo 2

Notes on Gaussian Elimination: 

In [87]:
# Swap two numpy arrays
def swapRow(rref, row1, row2):
    # print("Swap Row {} and Row {}".format(row1, row2))
    rref[[row1, row2]] = rref[[row2, row1]]

# Subtract one numpy array from another (in mod 2)
def addRow(rref, redRow, leadRow):
    # print("Subtracting Lead Row {} from Row {}".format(leadRow, redRow))
    rref[redRow] = np.absolute(np.subtract(rref[redRow], rref[leadRow]))

# Search for 1's above the leading element in current row 
def searchAbove(rref, col, leadRow):
    # print("Serch Above {} in col {}".format(leadRow,col))
    # print(rref)
    # Top-down row checking
    row = 0
    if (row == leadRow):
        # print("Search Failed: Lead Row {} Equal to Starting Search Row {}".format(leadRow, row))
        return -1
    while (row < leadRow):
        if (rref[row][col]):
            # print("Subtracting above")
            # print(rref)
            addRow(rref, row, leadRow)
            # print("After Subtract")
            # print(rref)
        row = row + 1
    return 0

# Search for 1's below the leading element in current row 
def searchBelow(rref, col, leadRow):
    # Down-Up row checking
    # print("Serch Below")
    # print(rref)
    row = rref.shape[0] - 1
    if (row == leadRow):
        # print("Search Failed: Lead Row {} Equal to Starting Search Row {}".format(leadRow, row))
        return -1
    while (row > leadRow):
        if (rref[row][col]):
            # print("Subtracting below")
            # print(rref)
            addRow(rref, row, leadRow)
            # print("After Subtract")
            #print(rref)
        row = row - 1
    return 0

# If leading element in row is not 1, search for new leading row 
def searchLead(rref, row, col):
    newRow, newCol = row + 1, col
    maxRow, maxCol = rref.shape[0], rref.shape[1]
    foundLead = 0
    # Ensure to not overflow dims of matrix
    if (newRow == maxRow or newCol == maxCol):
        # print("Maximum Row or Column Reached: [Row: {}| Column: {}]".format(maxRow, maxCol))
        return -1, -1
    while (newCol < maxCol):
        newRow = row + 1
        while (newRow < maxRow):
            if (rref[newRow][newCol]):
                # Found a leading 1 => break and return 
                foundLead = 1
                break
            newRow = newRow + 1
        if (foundLead):
            break
        newCol = newCol + 1
    if (not(foundLead)):
        print("New Leading Element not Found")
    return newRow, newCol

# GAUSSIAN ELIMINATION
# mat = matrix to be row reduced
# Returns rref, the input matrix in reduced row-echelon form
def gaussElim(mat):
    # METHOD:
    # Start at column 0
    # For row in row:
    #   if row, col value == 0
    #       loop over column values (below) in search of a leading zero (if nothing in that colmn then go to next column) - return its row index
    #       swap row with current one
    #       set current column if there were no more 1's in that column
    #   Otherwise, check all rows above and below
    #       Implement function to loop over above and subtract current row from it if a 1
    #               Handle out of bounds case (i.e., if it is a first row)
    #       Implement function to loop over below and subract current row from it if a 1
    #               Handle out of bounds case if last row 
    #  Increment column
    rref = np.asarray(mat)
    rows, cols = rref.shape[0], rref.shape[1]
    row, col = 0, 0
    while (row < rows):
        # Check Lead
        if (rref[row][col] == 0):
            newRow, col = searchLead(rref, row, col)
            if (newRow == -1):
                break
            elif(newRow != row):
                # print("Swap")
                # print(rref)
                swapRow(rref, row, newRow)
                # print("After Swap")
                # print(rref)
                # print("New Pivot to check sub: [Row: {} | Col: {}]".format(row, col))
            else :
                print("Error Finding Lead")
                break
        # Begin reduction
        searchAbove(rref, col, row)
        searchBelow(rref, col, row)
        col = col + 1
        row = row + 1
        # print(rref)
        # Check column bounds
        if (col > cols):
            break
    return rref

In [88]:
test1 = [[0, 0, 1], [1, 0, 1], [1, 1, 0]]
print("Easy Test Result...")
print(gaussElim(test1))
test2 = [[1, 0, 1, 1, 1], [1, 0, 1, 1, 1], [1, 0, 0, 1, 0], [1, 0, 1, 0, 0]]
print("Medium Test...")
print(test2)
print(gaussElim(test2))
test3 = [[0,1,0,1,0,0,1,0,0,0],[0,0,1,1,0,1,0,0,1,1],[1,0,0,1,0,0,0,1,0,0],[1,0,1,0,0,1,0,1,1,1],[1,1,0,0,1,0,0,1,1,1],[1,0,0,0,1,1,1,1,1,0],[1,1,0,1,1,0,1,1,0,1],[1,0,0,1,0,0,0,0,0,0]]
print("Hard Test Result...")
print(gaussElim(test3))

Easy Test Result...
[[1 0 0]
 [0 1 0]
 [0 0 1]]
Medium Test...
[[1, 0, 1, 1, 1], [1, 0, 1, 1, 1], [1, 0, 0, 1, 0], [1, 0, 1, 0, 0]]
[[1 0 0 0 1]
 [0 0 1 0 1]
 [0 0 0 1 1]
 [0 0 0 0 0]]
Hard Test Result...
[[1 0 0 0 0 0 1 0 1 0]
 [0 1 0 0 0 0 0 0 1 0]
 [0 0 1 0 0 0 0 0 1 0]
 [0 0 0 1 0 0 1 0 1 0]
 [0 0 0 0 1 0 1 0 1 1]
 [0 0 0 0 0 1 1 0 1 1]
 [0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 0]]


The Basis for the Solution Matrix Found in Gaussian Elimination  
* Find all free variables (or columns without leading 1's)  
* Test all combinations of free variables (0 || 1)  
* Append Each vector 

In [85]:
# FIND BASIS FOR SOLUTION SET
# gauss = the rref matrix 
# Returns set, the set of independent vector solutions to the linear system
def basis(gauss):
    gauss = np.asarray(gauss)
    freeVars = [0] * gauss.shape[1]
    row, col = 0, 0
    maxRow, maxCol = gauss.shape[0], gauss.shape[1]
    # Search for leading zeros
    while (row < maxRow):
        while (col < maxCol):
            if (gauss[row][col]):
                # Found leading element
                freeVars[col] = 1
                col = col + 1
                break
            col = col + 1
        row = row + 1
    # Show all free variabls
    for i in range(len(freeVars)):
        if freeVars[i] == 0:
            print(i)  
    print(freeVars)
    return 0

In [86]:
test = [[1, 1, 0, 0, 1], [0, 0, 1, 1, 0], [0, 0, 0, 0, 1]]
print(test)
basis(test)

[[1, 1, 0, 0, 1], [0, 0, 1, 1, 0], [0, 0, 0, 0, 1]]
1
3
[1, 0, 1, 0, 1]


0

In [76]:
a = [0, 1, 0]
rref = np.asarray(a)
print(rref)
print(np.logical_not(rref))

[0 1 0]
[ True False  True]


### Part 4: Generate Factor Base

Before finding the factor base, need a means of getting all primes up to some integer, B  
Sieve of Eratosthenes:    
* Starting with p = 2, color all multiples of p up to b 
* For each iteration, the next non-colored integer is prime  
* Visual: https://en.wikipedia.org/wiki/Sieve_of_Eratosthenes


In [7]:
# Sieve of Eratosthenes
# b = Limit of base
# Returns primes, an array of primes up to B
def eratosthenes(b):
    r = int(math.ceil(math.sqrt(b)))
    p = 2
    primes = []
    nums = [0] * b
    nums[0] = 1
    num = 1
    while (True):
        loc = p
        while (True):
            loc = p * num
            if (loc > b):
                break
            nums[loc-1] = 1
            num = num + 1
        primes.append(p)
        num = 1
        # Find next prime 
        loc = p
        found = 0
        while (loc <= b):
            # Number not yet colored - next prime
            if (nums[loc-1] == 0):
                p = loc
                found = 1
                break
            loc = loc + 1
        # Sought up to root and no more primes - done
        if (found == 0):
            break  
    return primes

arr = eratosthenes(1000)
print(len(arr))
    

168


Notes on Factor Bases:  
* Find list of primes less than integer, B (via sieve of eratosthenes)  
* Ensure residues exist for n (number to be factored) and p (a prime element in the list)

In [8]:
# FACTOR BASES
# b = Upper prime bound
# n = number to be factored
# Returns base, a list of primes
def factorBase(b, n):
    base = []
    primes = eratosthenes(b)
    for p in primes:
        if (jacobi(n, p) == 1):
            base.append(p)
    return base

print(factorBase(30, 73))

[2, 3, 19, 23]


### Part 5: Sieve

Notes on Sieve Implementation:  
* M = B^2

## Examples of Factoring using the Quadratic Sieve

### Part 6: Wrap Up